In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import string

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['dev_server']
sladb = config['db']['sladb']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + sladb + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Write all of the sql queries required for the tests

In [5]:
sql_count_sla_id = 'select count(*) from sladb.dbo.tbl_ref_sla'

In [6]:
sql_distinct_sla_id = 'select distinct sla_id from sladb.dbo.tbl_ref_sla'

In [7]:
# sql_sla_code = '''select * from \
# sladb.dbo.tbl_ref_sla_code'''

In [8]:
sql_sla_code = '''select min(sla_code) as min_sla_code, 
                          max(sla_code) as max_sla_code, 
                          count(*) as n_sla_codes
                   from sladb.dbo.tbl_ref_sla_code'''

In [9]:
sql_date_category = 'select distinct(date_category_id) from sladb.dbo.tbl_ref_sla_season_category'

In [10]:
sql_sla_translation = 'select sla_id, sla_code, date_category_id from sladb.dbo.tbl_ref_sla_translation'

## Execute all of the sql queries and store the results

In [11]:
before_count_sla_id = pd.read_sql(con = engine, sql = sql_count_sla_id).values[0][0]

In [12]:
before_count_sla_id

4

In [13]:
before_squared_count_sla_id = before_count_sla_id * before_count_sla_id

In [14]:
before_squared_count_sla_id

16

In [15]:
before_distinct_sla_id = pd.read_sql(con = engine, sql = sql_distinct_sla_id)

In [16]:
before_distinct_sla_id

,sla_id
0,A
1,B
2,C
3,N


In [17]:
before_sla_code = pd.read_sql(con = engine, sql = sql_sla_code)

In [18]:
before_sla_code

,min_sla_code,max_sla_code,n_sla_codes
0,1,16,16


In [19]:
before_date_category = pd.read_sql(con = engine, sql = sql_date_category)

In [20]:
before_date_category

,date_category_id
0,1
1,2


In [21]:
before_sla_translation = pd.read_sql(con = engine, sql = sql_sla_translation)

In [22]:
before_sla_translation

,sla_id,sla_code,date_category_id
0,A,1,1
1,A,2,1
2,A,3,1
3,A,4,1
4,A,1,2
5,A,5,2
6,A,9,2
7,A,13,2
8,B,5,1
9,B,6,1


In [23]:
timing = []

In [24]:
descriptions = []

In [25]:
results = []

## Before Tests

### First before test

In [26]:
timing.append('before')

In [27]:
descriptions.append('''In tbl_ref_sla test that the count of sla_ids squared \
is equal to the number of sla_codes in tbl_ref_sla_code''')

In [28]:
results.append(before_squared_count_sla_id == before_sla_code['n_sla_codes'].values[0])

### Second before test

In [29]:
timing.append('before')

In [30]:
descriptions.append('''In tbl_ref_sla_code test that the number of sla_codes  \
is equal to the maximum sla_code in tbl_ref_sla_code''')

In [31]:
results.append(before_sla_code['n_sla_codes'].values[0] == before_sla_code['max_sla_code'].values[0])

### Third before test

In [32]:
timing.append('before')

In [33]:
descriptions.append('''In tbl_ref_sla_code test that the minimum sla_code \
is equal to 1''')

In [34]:
results.append(1 == before_sla_code['min_sla_code'].values[0])

### Fourth before test

In [35]:
timing.append('before')

In [36]:
descriptions.append('''Test that the count of records in \
tbl_ref_sla_translation is equal to the count of 'sla_code's \
in tbl_ref_sla_code multiplied by the number of 'date_category_id's \
in tbl_ref_sla_season_category''')

In [37]:
results.append(
    len(before_sla_translation) == (before_sla_code['n_sla_codes'].values[0] *
                                    len(before_date_category)))

### Fifth before test

In [38]:
timing.append('before')

In [39]:
descriptions.append('''Test the uniqueness of the rows \
in tbl_ref_sla_translation''')

In [40]:
before_sla_translation_nodup = before_sla_translation.drop_duplicates()

In [41]:
results.append(len(before_sla_translation) == len(before_sla_translation_nodup))

### Sixth before test

In [42]:
timing.append('before')

In [43]:
descriptions.append('''Test that all sla_ids, sla_codes and \
date_category_ids have been accounted for in tbl_ref_sla_translation''')

In [44]:
before_unique_sla_translation = before_sla_translation.nunique()

In [45]:
results.append(
    ((before_unique_sla_translation['sla_id'] == before_count_sla_id)
     and (before_unique_sla_translation['date_category_id'] == len(
         before_date_category))
     and (before_unique_sla_translation['sla_code'] ==
          before_sla_code['n_sla_codes'].values[0])))

# Insert the sla_id values into tbl_ref_sla so that the after tests can be conducted

In [46]:
ninserts = 2

In [47]:
sla_id = [x for x in string.ascii_uppercase if x not in(before_distinct_sla_id.values)]

In [48]:
sla_inserts = [[x, 'Testing new SLA: '+ x, 1, 2] for x in sla_id]

In [49]:
sla_inserts_df = pd.DataFrame(sla_inserts[0:ninserts], 
                              columns = ['sla_id', 'sla_desc', 'sla_min_days', 'sla_max_days'])

In [50]:
sla_inserts_df

,sla_id,sla_desc,sla_min_days,sla_max_days
0,D,Testing new SLA: D,1,2
1,E,Testing new SLA: E,1,2


In [51]:
# sla_inserts_df.to_sql('tbl_ref_sla', con = engine, index = False, if_exists = 'append')

## Execute all of the sql queries AFTER INSERT and store the results

In [52]:
after_count_sla_id = pd.read_sql(con = engine, sql = sql_count_sla_id).values[0][0]

In [53]:
after_count_sla_id

4

In [54]:
after_squared_count_sla_id = (after_count_sla_id)**2 

In [55]:
after_squared_count_sla_id

16

In [56]:
after_distinct_sla_id = pd.read_sql(con = engine, sql = sql_distinct_sla_id)

In [57]:
after_distinct_sla_id

,sla_id
0,A
1,B
2,C
3,N


In [58]:
after_sla_code = pd.read_sql(con = engine, sql = sql_sla_code)

In [59]:
after_sla_code

,min_sla_code,max_sla_code,n_sla_codes
0,1,16,16


In [60]:
after_date_category = pd.read_sql(con = engine, sql = sql_date_category)

In [61]:
# after_date_category

In [62]:
after_sla_translation = pd.read_sql(con = engine, sql = sql_sla_translation)

In [63]:
# after_sla_translation

## After Tests

### First after test

In [64]:
timing.append('after')

In [65]:
descriptions.append('''Check that the count of sla_ids (from tbl_ref_sla) squared \
is equal to the number of sla_codes in tbl_ref_sla_code''')

In [66]:
results.append(after_squared_count_sla_id == after_sla_code['n_sla_codes'].values[0])

In [67]:
results

[True, True, True, True, True, True, True]

### Second after test

In [68]:
timing.append('after')

In [69]:
descriptions.append('''Test that the number of sla_codes in tbl_ref_sla_code \
is equal to the maximum sla_code in tbl_ref_sla_code''')

In [70]:
results.append(after_sla_code['n_sla_codes'].values[0] == after_sla_code['max_sla_code'].values[0])

### Third after test

In [71]:
timing.append('after')

In [72]:
descriptions.append('''Test that the minimum sla_code in tbl_ref_sla_code is equal to 1''')

In [73]:
results.append(1 == after_sla_code['min_sla_code'].values[0])

### Fourth after test

In [74]:
timing.append('after')

In [75]:
descriptions.append(
    '''Test that the count of records in tbl_ref_sla_translation is \
    equal to the count of sla_codes in tbl_ref_sla_code multiplied by \
    the number of date_category_ids in tbl_ref_sla_season_category''')

In [76]:
results.append(len(after_sla_translation) == (after_sla_code['n_sla_codes'].values[0] * len(after_date_category)))

### Fifth before test

In [77]:
timing.append('after')

In [78]:
descriptions.append('''Test the uniqueness of the rows in tbl_ref_sla_translation''')

In [79]:
after_sla_translation_nodup = after_sla_translation.drop_duplicates()

In [80]:
results.append(len(after_sla_translation) == len(after_sla_translation_nodup))

### Sixth after test

In [81]:
timing.append('after')

In [82]:
descriptions.append('''Test that all sla_ids, sla_codes and date_category_ids have been\
accounted for in tbl_ref_sla_translation''')

In [83]:
after_unique_sla_translation = after_sla_translation.nunique()

In [84]:
results.append(
    ((after_unique_sla_translation['sla_id'] == after_count_sla_id)
     and (after_unique_sla_translation['date_category_id'] == len(
         after_date_category)) and (after_unique_sla_translation['sla_code'] ==
                                    after_sla_code['n_sla_codes'].values[0])))

### Seventh after test

In [85]:
timing.append('after')

In [86]:
m = ninserts
n = before_count_sla_id

In [87]:
descriptions.append(
    '''Test that the number of rows insrted into tbl_ref_sla_code\
    is equal to (m+n)**2-n**2 '''
)

In [88]:
results.append(after_sla_code.n_sla_codes.values[0] -
               before_sla_code.n_sla_codes.values[0] == (m + n)**2 - n**2)

### Eighth after test

In [89]:
timing.append('after')

In [90]:
descriptions.append(
    '''Test that the nmuber of rows in tbl_ref_sla_translation is equal to 2*(m+n)**2 '''
)

In [91]:
num_date_category_ids = len(after_date_category)

In [92]:
results.append(len(after_sla_translation) == num_date_category_ids * (m + n)**2)

### Ninth after test

In [93]:
timing.append('after')

In [94]:
descriptions.append('''After removing the original rows, \
test the uniqueness of the rows in tbl_ref_sla_translation''')

In [95]:
after_sla_translation_nodup = after_sla_translation.drop_duplicates()

In [96]:
results.append(len(after_sla_translation) == len(after_sla_translation_nodup))

### Tenth after test

In [97]:
timing.append('after')

In [98]:
descriptions.append(
    '''Ensuring the correct number of rows was inserted into tbl_ref_sla_trnalsation'''
)

In [99]:
added_rows_df = pd.merge(after_sla_translation,
         before_sla_translation,
         on=['sla_id', 'sla_code', 'date_category_id'],
         indicator=True,
         how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

In [100]:
results.append(len(added_rows_df)==(len(after_sla_translation) - len(before_sla_translation)))

## List of tuples with final test results

In [101]:
list(zip(timing, descriptions, results))

[('before',
  'In tbl_ref_sla test that the count of sla_ids squared is equal to the number of sla_codes in tbl_ref_sla_code',
  True),
 ('before',
  'In tbl_ref_sla_code test that the number of sla_codes  is equal to the maximum sla_code in tbl_ref_sla_code',
  True),
 ('before',
  'In tbl_ref_sla_code test that the minimum sla_code is equal to 1',
  True),
 ('before',
  "Test that the count of records in tbl_ref_sla_translation is equal to the count of 'sla_code's in tbl_ref_sla_code multiplied by the number of 'date_category_id's in tbl_ref_sla_season_category",
  True),
 ('before',
  'Test the uniqueness of the rows in tbl_ref_sla_translation',
  True),
 ('before',
  'Test that all sla_ids, sla_codes and date_category_ids have been accounted for in tbl_ref_sla_translation',
  True),
 ('after',
  'Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of sla_codes in tbl_ref_sla_code',
  True),
 ('after',
  'Test that the number of sla_codes in tbl_ref_sla

In [102]:
pd.DataFrame(list(zip(timing, descriptions, results)), columns = {'before/after', 'test_desc', 'result'})             

,result,test_desc,before/after
0,before,In tbl_ref_sla test that the count of sla_ids ...,True
1,before,In tbl_ref_sla_code test that the number of sl...,True
2,before,In tbl_ref_sla_code test that the minimum sla_...,True
3,before,Test that the count of records in tbl_ref_sla_...,True
4,before,Test the uniqueness of the rows in tbl_ref_sla...,True
5,before,"Test that all sla_ids, sla_codes and date_cate...",True
6,after,Check that the count of sla_ids (from tbl_ref_...,True
7,after,Test that the number of sla_codes in tbl_ref_s...,True
8,after,Test that the minimum sla_code in tbl_ref_sla_...,True
9,after,Test that the count of records in tbl_ref_sla_...,True
